In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
train = pd.read_csv('/kaggle/input/choose-tutors/train.csv')
test = pd.read_csv('/kaggle/input/choose-tutors/test.csv')

# train = pd.read_csv('train.csv', index_col='Id')
# test = pd.read_csv('test.csv', index_col='Id')

In [4]:
train.shape, test.shape

In [5]:
train.head()

In [6]:
train.columns

In [7]:
features_name = ['age', 'years_of_experience', 'lesson_price', 'qualification',
            'physics', 'chemistry', 'biology', 'english', 'geography', 'history',
            'mean_exam_points']
target_name = 'choose'
target = train[target_name]

In [8]:
train.describe()

In [9]:
cat_features = ['years_of_experience', 'qualification']
num_features = ['age', 'lesson_price', 'mean_exam_points']
bin_features = ['physics', 'chemistry', 'biology', 'english', 'geography', 'history']

### Реализация алгоритма RandomForestClassifier

In [10]:
class Node:
    """Класс узла"""
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [11]:
class Leaf:
    """Класс терминального узла (листа)"""
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        #  найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        prediction = max(classes, key=classes.get)
        return prediction

In [12]:
# Расчет критерия Джини

def gini(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    impurity = 1
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2
        
    return impurity

In [13]:
# Расчет энтропийного критерия

def entropy(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1

    #  расчет критерия
    impurity = 1
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p * (np.log(p) / np.log(2))

    return impurity

In [14]:
class RandomForestClassifier():
    """Класс алгоритма RandomForestClassifier"""
    
    @property
    def forest_(self):
        return self.forest
    
    @property
    def n_trees_(self):
        return self.n_trees

    def __init__(self, n_trees = 1, max_depth=5, min_leaf=1, random_state=42, criterion = 'gini'):
        self.n_trees = n_trees
        self.random_state = random_state
        self.max_depth = max_depth
        self.min_leaf = min_leaf
        self.forest = []
        self.criterion = criterion

    def bootstrap(self, X, y):
        n_samples = X.shape[0]
        np.random.seed(self.random_state)
        
        bootstrap = []
        for i in range(self.n_trees):
            b_data = np.zeros(X.shape)
            b_labels = np.zeros(y.shape)
            
            for j in range(n_samples):
                sample_index = np.random.randint(0, n_samples-1)
                b_data[j] = X[sample_index]
                b_labels[j] = y[sample_index]
            bootstrap.append((b_data, b_labels))
        
        return bootstrap
    
    def subsample(self, len_sample):
        # будем сохранять не сами признаки, а их индексы
        sample_indexes = [i for i in range(len_sample)]
        
        len_subsample = int(np.sqrt(len_sample))
        
        subsample = []
        np.random.shuffle(sample_indexes)
        for _ in range(len_subsample):
            subsample.append(sample_indexes.pop())
        
        return subsample
    
    def calc_criterion(self, y):
        """Расчет критерия информативности"""

        #  подсчет количества объектов разных классов
        classes = {}
        for label in y:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1

        impurity = 1
        if self.criterion == 'gini':
            # Расчет критерия Джини
            for label in classes:
                p = classes[label] / len(y)
                impurity -= p ** 2
        elif self.criterion == 'entropy':
            # Расчет энтропийного критерия
            for label in classes:
                p = classes[label] / len(labels)
                impurity -= p * (np.log(p) / np.log(2))

        return impurity
    
    def quality(self, left_labels, right_labels, current_criterion):
        """Расчет качества"""

        # доля выбоки, ушедшая в левое поддерево
        p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
        
        return current_criterion - p * self.calc_criterion(left_labels) - (1 - p) * self.calc_criterion(right_labels)

    @staticmethod
    def split(X, y, index, t):
        """Разбиение датасета в узле"""

        left = np.where(X[:, index] <= t)
        right = np.where(X[:, index] > t)
            
        true_data = X[left]
        false_data = X[right]
        true_labels = y[left]
        false_labels = y[right]
            
        return true_data, false_data, true_labels, false_labels

    def find_best_split(self, X, y):
        """Нахождение наилучшего разбиения"""

        current_criterion = self.calc_criterion(y)

        best_quality = 0
        best_t = None
        best_index = None
        
        n_features = X.shape[1]
        
        # выбор индекса из подвыборки длиной sqrt(n_features)
        subsample = self.subsample(n_features)
        
        for index in subsample:
            # будем проверять только уникальные значения признака, исключая повторения
            t_values = np.unique([row[index] for row in X])
            
            for t in t_values:
                true_data, false_data, true_labels, false_labels = self.split(X, y, index, t)
                #  пропускаем разбиения, в которых в узле объектов менее заданных в min_leaf
                if len(true_data) < self.min_leaf or len(false_data) < self.min_leaf:
                    continue
                
                current_quality = self.quality(true_labels, false_labels, current_criterion)
                
                #  выбираем порог, на котором получается максимальный прирост качества
                if current_quality > best_quality:
                    best_quality, best_t, best_index = current_quality, t, index

        return best_quality, best_t, best_index

    def tree(self, X, y):

        def build_tree(X, y, max_depth=None, _depth=0, **kwargs):
            """Построение дерева с помощью рекурсивной функции"""
            
            #  Базовый случай - прекращаем рекурсию, когда достигли максимальной глубины
            if max_depth and _depth >= max_depth:
                return Leaf(X, y)

            quality, t, index = self.find_best_split(X, y)

            #  Базовый случай - прекращаем рекурсию, когда нет прироста качества
            if quality == 0:
                return Leaf(X, y)

            true_data, false_data, true_labels, false_labels = self.split(X, y, index, t)

            # Рекурсивно строим два поддерева
            true_branch = build_tree(true_data, true_labels, depth=kwargs['depth'])
            false_branch = build_tree(false_data, false_labels, depth=kwargs['depth'])
            
            # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
            return Node(index, t, true_branch, false_branch)
    
        return build_tree(X, y, depth=0)
    
    def predict(self, X, proba=False):
        """Предсказание голосованием деревьев """
        
        def tree_predict(X, tree):
            """Функция формирования предсказания по выборке на одном дереве"""
            
            return [classify_object(obj, tree) for obj in X]
        
        def classify_object(obj, node):
            """Функция классификации отдельного объекта"""

            #  Останавливаем рекурсию, если достигли листа
            if isinstance(node, Leaf):
                return node.prediction
            
            if obj[node.index] <= node.t:
                return classify_object(obj, node.true_branch)
            else:
                return classify_object(obj, node.false_branch)
        
        # добавим предсказания всех деревьев в список
        predictions = []
        for tree in self.forest:
            prediction = tree_predict(X, tree)
            predictions.append(prediction)
        
        # сформируем список с предсказаниями для каждого объекта
        predictions_per_object = list(zip(*predictions))
        
        answers = []
        if proba is True:
            for obj in predictions_per_object:
                class_0 = 0
                class_1 = 0
                for itm in obj:
                    if itm == 0:
                        class_0 += 1
                    else:
                        class_1 += 1
                probability = class_1 / (class_0 + class_1)
                answers.append(probability)
        else:
            for obj in predictions_per_object:
                predicted_class = max(set(obj), key=obj.count)
                answers.append(predicted_class)
        
        return answers

    def predict_proba(self, X):
        """Получение вероятности предсказания"""
        
        return self.predict(X, proba=True)
    
    def fit(self, X, y):
        bootstrap = self.bootstrap(X, y)
        
        for b_data, b_labels in bootstrap:
            self.forest.append(self.tree(b_data, b_labels))

### Вспомогательные функции

In [15]:
def error_matrix(y, y_pred):
    TP, TN, FP, FN = 0, 0, 0, 0
    for i in range(y.shape[0]):
        if y[i] > y_pred[i]:
            FN += 1
        elif y[i] < y_pred[i]:
            FP += 1
        elif y[i] == 1:
            TP += 1
        else:
            TN += 1
    return np.array([[FN, FP], [TN, TP]])  # порядок такой, чтобы обращение было "говорящим": TN = array[1][0]

In [16]:
def measures(y, y_pred, betta=1):
    e_m = error_matrix(y, y_pred)
    TP = e_m[1][1]
    TN = e_m[1][0]
    FP = e_m[0][1]
    FN = e_m[0][0]
    accuracy = np.mean(y == y_pred)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    FPR = FP / (FP + TN)
    F_measure = (1 + betta**2) * precision * recall / (betta**2 * precision + recall)
    return accuracy, precision, recall, FPR, F_measure

In [17]:
def roc_auc_score(y, y_pred):
    n = len(y)
    step_tpr = 1 / sum(y)
    step_fpr = 1 / (n - sum(y))
    
    df = pd.DataFrame({
        'y_true': y,
        'y_pred': y_pred
    })
    df = df.sort_values(by='y_pred', ascending=False).reset_index(drop=True)
    tprs = [0]
    fprs = [0]
    auc = 0
    for i in range(n):
        if df['y_true'][i] == 1:
            tprs.append(tprs[-1] + step_tpr)
            fprs.append(fprs[-1])
        elif df['y_true'][i] == 0:
            tprs.append(tprs[-1])
            fprs.append(fprs[-1] + step_fpr)
            auc += tprs[-1] * step_fpr
    return auc, tprs, fprs

In [18]:
def standardization(x):
    """Функция стандартизации"""
    
    return (x - x.mean()) / x.std()

In [19]:
def print_results(model, X_train, X_test, y_train, y_test):

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    print('Train')
    accuracy, precision, recall, FPR, F_measure = measures(y_train, y_train_pred)
    auc_score = roc_auc_score(y_train, model.predict_proba(X_train))[0]
    print(f'Accuracy: {accuracy:.3f}')
    print(f'ROC AUC: {auc_score:.3f}')
    
    print('\nTest')
    accuracy, precision, recall, FPR, F_measure = measures(y_test, y_test_pred)
    auc_score = roc_auc_score(y_test, model.predict_proba(X_test))[0]
    print(f'Accuracy: {accuracy:.3f}')
    print(f'ROC AUC: {auc_score:.3f}')

### Разбиение на train/test

In [20]:
X = np.array(train[features_name])
y = np.array(train[target_name].astype(int))

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    shuffle=True,
    test_size=0.25,
    random_state=0,
    stratify=y
)

### Обучение модели

In [21]:
n_treeses=[10, 50, 100, 300, 500]
max_depths=[3, 5, 7, 10]
min_leafs=[1, 3, 5, 7, 10, 20, 50]

In [22]:
combinations = [(n_trees, max_depth, min_leaf)
               for n_trees in n_treeses
               for max_depth in max_depths
               for min_leaf in min_leafs]

In [23]:
best_model = RandomForestClassifier(
    n_trees=300,
    max_depth=3,
    min_leaf=7,
    random_state=0,
)
best_model.fit(X_train, y_train)

In [24]:
print_results(best_model, X_train, X_test, y_train, y_test)

### Сохранение результата

In [25]:
X_final = np.array(test)
y_pred_proba_final = best_model.predict_proba(X_final)

preds_final = pd.DataFrame()
preds_final['Id'] = test['Id'].copy()
preds_final['choose'] = y_pred_proba_final
preds_final.to_csv('./predictions.csv', index=False, encoding='utf-8', sep=',')

preds_final.head()